In [139]:
import os.path
from os import path

import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from sklearn.model_selection import train_test_split

In [140]:
print(tf.__version__)

2.1.0


In [141]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"

csv_file = keras.utils.get_file(fname="wdbc.data", origin=DATA_URL)

In [166]:
# Make numpy values easier to read.
np.set_printoptions(precision=6, suppress=True)

In [174]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv(csv_file)

df.columns = ['id', 'diagnosis', 'radius', 'texture', 'perimeter', 'area', 'smoothness','compactness', 'concavity',
              'concave_points', 'symmetry', 'fractal_dimension', 'radius_se','texture_se', 'perimeter_se', 'area_se',
              'smoothness_se', 'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se', 'fractal_dimension_se',
              'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst',
              'concavity_worst', 'concave_points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df.diagnosis.replace(to_replace=['M', 'B'], value=[1, 0], inplace=True)

df.head()

,id,diagnosis,radius,texture,perimeter,area,smoothness,compactness,concavity,concave_points,symmetry,fractal_dimension,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
1,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
2,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
3,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678
4,843786,1,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.03345,0.03672,0.01137,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440


### Dados normalizados

In [169]:
diagnosis = df.pop('diagnosis')
id_dataset = df.pop('id')

column_maxes = df.max()
df_max = column_maxes.max()
column_mins = df.min()
df_min = column_mins.min()
normalized_df = (df - df_min) / (df_max - df_min)

normalized_df.head()

,radius,texture,perimeter,area,smoothness,compactness,concavity,concave_points,symmetry,fractal_dimension,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,0.004835,0.004177,0.031241,0.311707,0.000020,0.000018,0.000020,0.000016,0.000043,0.000013,0.000128,0.000173,0.000799,0.017414,0.000001,0.000003,0.000004,0.000003,0.000003,8.302774e-07,0.005874,0.005503,0.037330,0.459803,0.000029,0.000044,0.000057,0.000044,0.000065,0.000021
1,0.004629,0.004995,0.030559,0.282793,0.000026,0.000038,0.000046,0.000030,0.000049,0.000014,0.000175,0.000185,0.001078,0.022104,0.000001,0.000009,0.000009,0.000005,0.000005,1.074518e-06,0.005541,0.006001,0.035849,0.401740,0.000034,0.000100,0.000106,0.000057,0.000085,0.000021
2,0.002685,0.004791,0.018237,0.090762,0.000033,0.000067,0.000057,0.000025,0.000061,0.000023,0.000117,0.000272,0.000810,0.006401,0.000002,0.000018,0.000013,0.000004,0.000014,2.164551e-06,0.003505,0.006229,0.023242,0.133451,0.000049,0.000204,0.000161,0.000061,0.000156,0.000041
3,0.004770,0.003371,0.031758,0.304890,0.000024,0.000031,0.000047,0.000025,0.000043,0.000014,0.000178,0.000184,0.001278,0.022200,0.000003,0.000006,0.000013,0.000004,0.000004,1.202398e-06,0.005299,0.003919,0.035778,0.370240,0.000032,0.000048,0.000094,0.000038,0.000056,0.000018
4,0.002927,0.003691,0.019410,0.112153,0.000030,0.000040,0.000037,0.000019,0.000049,0.000018,0.000079,0.000209,0.000521,0.006392,0.000002,0.000008,0.000009,0.000003,0.000005,1.194640e-06,0.003637,0.005583,0.024307,0.174330,0.000042,0.000123,0.000126,0.000041,0.000094,0.000029


In [170]:
dataset = tf.data.Dataset.from_tensor_slices((normalized_df.values, diagnosis.values))

In [172]:
def show_batch(dataset):
    for batch, label in dataset.take(2):
        print ('Features: {}, Diagnosis: {}'.format(batch.numpy(), label))

In [173]:
print("Diagnosis: 0-Benign, 1-Malign")
show_batch(dataset)

Diagnosis: 0-Benign, 1-Malign
Features: [0.004835 0.004177 0.031241 0.311707 0.00002  0.000018 0.00002  0.000016
 0.000043 0.000013 0.000128 0.000173 0.000799 0.017414 0.000001 0.000003
 0.000004 0.000003 0.000003 0.000001 0.005874 0.005503 0.03733  0.459803
 0.000029 0.000044 0.000057 0.000044 0.000065 0.000021], Diagnosis: 1
Features: [0.004629 0.004995 0.030559 0.282793 0.000026 0.000038 0.000046 0.00003
 0.000049 0.000014 0.000175 0.000185 0.001078 0.022104 0.000001 0.000009
 0.000009 0.000005 0.000005 0.000001 0.005541 0.006001 0.035849 0.40174
 0.000034 0.0001   0.000106 0.000057 0.000085 0.000021], Diagnosis: 1
